<a href="https://colab.research.google.com/github/snpsuen/Deep_Learning_Data/blob/main/script/Pytorch_MLP_binary_classification_JB_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# check pytorch version
import torch
print("pytorch version: " + torch.__version__)
!python --version

pytorch version: 2.5.1+cu121
Python 3.11.11


In [2]:
# pytorch mlp for binary classification
from numpy import vstack
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [3]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.nn import BCELoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_

In [4]:
# dataset definition
class CSVDataset(Dataset):
    # load the dataset
    def __init__(self, path):
        # load the csv file as a dataframe
        df = read_csv(path, header=None)
        # store the inputs and outputs
        self.X = df.values[:, :-1]
        self.y = df.values[:, -1]
        # ensure input data is floats
        self.X = self.X.astype('float32')
        # label encode target and ensure the values are floats
        self.y = LabelEncoder().fit_transform(self.y)
        self.y = self.y.astype('float32')
        self.y = self.y.reshape((len(self.y), 1))

        print("Type of Dataset.X = ", type(self.X))
        print("Type of Dataset.y = ", type(self.y))
        print("Dataset.X.shape = ", self.X.shape)
        print("Dataset.y.shape = ", self.y.shape)

    # number of rows in the dataset
    def __len__(self):
        return len(self.X)

    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]

    # get indexes for train and test rows
    def get_splits(self, n_test=0.33):
        # determine sizes
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])

    # get number of input features from X
    # def get_number_input_features(self):
    #    return self.X.shape[1]

In [6]:
# model definition
class MLP(Module):
    # define model elements
    def __init__(self, n_inputs, n_outputs):
        super(MLP, self).__init__()
        # input to first hidden layer
        self.hidden1 = Linear(n_inputs, 10)
        kaiming_uniform_(self.hidden1.weight, nonlinearity='relu')
        self.act1 = ReLU()
        # second hidden layer
        self.hidden2 = Linear(10, 8)
        kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
        self.act2 = ReLU()
        # third hidden layer and output
        self.hidden3 = Linear(8, n_outputs)
        xavier_uniform_(self.hidden3.weight)
        self.act3 = Sigmoid()

    # forward propagate input
    def forward(self, X):
        # input to first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)
        # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        # third hidden layer and output
        X = self.hidden3(X)
        X = self.act3(X)
        return X

In [16]:
# prepare train, test datasets
def prepare_train_test_datasets(path):
    # load the dataset
    dataset = CSVDataset(path)
    n_inputs = dataset.X.shape[1]
    n_outputs = dataset.y.shape[1]
    # calculate split
    train, test = dataset.get_splits()
    # get number of input features
    # n_inputs = dataset.get_number_input_features()
    return train, test, n_inputs, n_outputs

In [17]:
# prepare train, test data loaders
def prepare_train_test_dataloaders(train, test):
    train_dl = DataLoader(train, batch_size=32, shuffle=True)
    test_dl = DataLoader(test, batch_size=1024, shuffle=False)
    return train_dl, test_dl

In [18]:
# train the model
def train_model(train_dl, model):
    # define the optimization
    criterion = BCELoss()
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
    # enumerate epochs
    for epoch in range(100):
        # enumerate mini batches
        for i, (inputs, targets) in enumerate(train_dl):
            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            yhat = model(inputs)
            # calculate loss
            loss = criterion(yhat, targets)
            # credit assignment
            loss.backward()
            # update model weights
            optimizer.step()

In [19]:
# evaluate the model
def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        print("testdl iteration i: ", i)
        print("Type of testdl inputs = ", type(inputs))
        print("Type of testdl targets = ", type(targets))
        print("testdl inputs.shape = ", inputs.shape)
        print("testdl targets.shape = ", targets.shape)

        # evaluate the model on the test set
        yhat = model(inputs)
        # retrieve numpy array
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        actual = actual.reshape((len(actual), 1))
        # round to class values
        yhat = yhat.round()

        print("Type of testdl yhat = ", type(yhat))
        print("Type of testdl actual = ", type(actual))
        print("yhat.shape = ", yhat.shape)
        print("actual.shape = ", actual.shape)

        # store
        predictions.append(yhat)
        actuals.append(actual)
        print("Type of testdl predictions = ", type(predictions))
        print("Type of testdl actuals = ", type(actuals))

    predictions, actuals = vstack(predictions), vstack(actuals)
    # calculate accuracy
    acc = accuracy_score(actuals, predictions)
    return acc

In [20]:
# make a class prediction for one row of data
def predict(row, model):
    # convert row to data
    row = Tensor([row])
    # make prediction
    yhat = model(row)
    # retrieve numpy array
    yhat = yhat.detach().numpy()
    return yhat

In [21]:
# prepare the data
path = 'https://raw.githubusercontent.com/snpsuen/Deep_Learning_Data/refs/heads/main/dataset/ionosphere.csv'
train, test, n_inputs, n_outputs = prepare_train_test_datasets(path)
train_dl, test_dl = prepare_train_test_dataloaders(train, test)
print(len(train_dl.dataset), len(test_dl.dataset))
print("Number of input features = ", n_inputs)
print("Number of output targets = ", n_outputs)

Type of Dataset.X =  <class 'numpy.ndarray'>
Type of Dataset.y =  <class 'numpy.ndarray'>
Dataset.X.shape =  (351, 34)
Dataset.y.shape =  (351, 1)
235 116
Number of input features =  34
Number of output targets =  1


In [22]:
# define the network
model = MLP(n_inputs, n_outputs)

In [23]:
# train the model
train_model(train_dl, model)

In [24]:
# evaluate the model
acc = evaluate_model(test_dl, model)
print('Accuracy: %.3f' % acc)

testdl iteration i:  0
Type of testdl inputs =  <class 'torch.Tensor'>
Type of testdl targets =  <class 'torch.Tensor'>
testdl inputs.shape =  torch.Size([116, 34])
testdl targets.shape =  torch.Size([116, 1])
Type of testdl yhat =  <class 'numpy.ndarray'>
Type of testdl actual =  <class 'numpy.ndarray'>
yhat.shape =  (116, 1)
actual.shape =  (116, 1)
Type of testdl predictions =  <class 'list'>
Type of testdl actuals =  <class 'list'>
Accuracy: 0.888


In [25]:
# make a single prediction (expect class=1)
row = [1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1,0.03760,0.85243,-0.17755,0.59755,-0.44945,0.60536,-0.38223,0.84356,-0.38542,0.58212,-0.32192,0.56971,-0.29674,0.36946,-0.47357,0.56811,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300]
yhat = predict(row, model)
print('Predicted: %.3f (class=%d)' % (yhat, yhat.round()))

Predicted: 0.997 (class=1)


<ipython-input-25-08961a1e2401>:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('Predicted: %.3f (class=%d)' % (yhat, yhat.round()))
